In [93]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd
import datetime as dt
load_dotenv()

True

In [94]:
TOKEN = os.getenv('TOKEN')
try:
    news = pd.read_csv("../data/News/news.csv")
except FileNotFoundError:
    news = pd.DataFrame(columns=["symbol","date","time","uuid","match","sentiment","content"])
API_site = "https://api.marketaux.com/v1/news/all"
others_setting = "filter_entities=true&language=en"
symbols = ["META", "AMZN", "AAPL", "NFLX", "GOOGL"]
stocknews = dict()

In [95]:
if news.shape[0] == 0:
    last_day = 0
else :
    last_day = news["date"].sort_values(ascending=True).iloc[0]
    last_day = dt.date.today() - dt.datetime.strptime(last_day,  "%Y-%m-%d").date()
    last_day = last_day.days
print(last_day)

31


In [96]:
def update_news(news, stocknews):
    news_dict = dict()
    for symbol, stocknew in stocknews.items():
        if "error" in stocknew:
            return news
        for new_info in stocknew["data"]:
            news_dict["symbol"] = symbol
            news_dict["uuid"] = uuid = new_info["uuid"]
            news_dict["content"] = new_info["title"]
            time = dt.datetime.strptime(new_info["published_at"], '%Y-%m-%dT%H:%M:%S.000000Z')
            news_dict["date"] = dt.datetime.strftime(time, "%Y-%m-%d")
            news_dict["time"] = dt.datetime.strftime(time, "%H:%M:%S")
            score = new_info["entities"][0]
            news_dict["match"] = score["match_score"]
            news_dict["sentiment"] = score["sentiment_score"]
            df_news_dict = pd.DataFrame([news_dict])
            if not news["uuid"].str.contains(uuid).any():
                news = pd.concat([news, df_news_dict], ignore_index=True)
    return news

In [97]:
for day in range(last_day, last_day+18):
    yesterday = dt.datetime.now() - dt.timedelta(day)
    yesterday = dt.datetime.strftime(yesterday, '%Y-%m-%d')
    for symbol in symbols:
        reqstr = f'{API_site}?symbols={symbol}&{others_setting}&published_before={yesterday}&api_token={TOKEN}'
        data = requests.get(reqstr, timeout=5)
        stocknews[symbol] = json.loads(data.text)
    news = update_news(news, stocknews)

In [98]:
print(news)
news.to_csv("../data/News/news.csv", index=0)

    symbol        date      time                                  uuid  \
0     META  2022-11-02  15:00:11  92bc08cd-1214-4b13-9044-4c32beaa86c3   
1     META  2022-11-02  15:00:10  02716d04-b261-4448-b641-d89af99abebc   
2     META  2022-11-02  13:54:28  50f79d6a-2035-4cd6-967a-aca7a520d94f   
3     AMZN  2022-11-02  15:18:14  a8ea05b4-cfa7-4af5-b2db-6fc7973db70e   
4     AMZN  2022-11-02  14:27:22  7d45e0f0-3f23-48fa-9e64-b2a60053a186   
..     ...         ...       ...                                   ...   
493   AAPL  2022-09-24  17:45:00  4afbca91-4368-4785-b7c8-273fa090781b   
494   NFLX  2022-09-24  05:50:10  9d934604-fcec-4bc1-aea4-4486b3a7007e   
495  GOOGL  2022-09-24  18:01:00  a6ac1a1d-2a95-4dd2-be8b-4b10c2b700cb   
496  GOOGL  2022-09-24  14:50:11  ef18c12a-5ce5-4a04-9196-42539be3b542   
497  GOOGL  2022-09-24  14:00:49  65e07bb6-7684-4c8d-b452-72a7e855582a   

         match  sentiment                                            content  
0    14.812694   0.671900  FAGAN